In [24]:
# improt the required libraries
import pandas as pd
import string
from nltk.corpus import stopwords

In [2]:
#read data from csv file 
sentiments_data = pd.read_csv('C:\Simplilearn\Data-Science-Python\Projects\Project-09\Lesson 9 - 2\imdb_labelled.txt',sep='\t',names = ['comments','labels'])

In [5]:
# view top ten
sentiments_data.head(10)

,comments,labels
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
5,"The rest of the movie lacks art, charm, meanin...",0
6,Wasted two hours.,0
7,Saw the movie today and thought it was a good ...,1
8,A bit predictable.,0
9,Loved the casting of Jimmy Buffet as the scien...,1


In [6]:
#view data statistics using describe()
sentiments_data.describe()

,labels
count,748.000000
mean,0.516043
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
#view columns of the dataset
sentiments_data.columns

Index(['comments', 'labels'], dtype='object')

In [8]:
# view more informations
sentiments_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  748 non-null    object
 1   labels    748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [9]:
sentiments_data.groupby('labels').describe()

comments                                              
          count unique                               top freq
labels                                                       
0           362    361                Not recommended.      2
1           386    384  Definitely worth checking out.      2

In [12]:
#verify the length of message and add it as a column
sentiments_data['length'] = sentiments_data['comments'].apply(len)



In [15]:
#verify the data
sentiments_data.head(5)

,comments,labels,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [16]:
# view first
sentiments_data[sentiments_data['length']>50]['comments'].iloc[0]

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '

In [19]:
#start with vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer =  CountVectorizer()

In [25]:
#creat a function to remove the stopwords
def removestopwords(messg):
    no_ponctuation = [char for char in messg if char not in string.punctuation]
    no_ponctuation = ''.join(no_ponctuation)
    return [word for word in no_ponctuation.split() if word.lower() not in stopwords.words('english')] 
# apply the function to the data
sentiments_data['comments'].head(5).apply(removestopwords)

0    [slowmoving, aimless, movie, distressed, drift...
1    [sure, lost, flat, characters, audience, nearl...
2    [Attempting, artiness, black, white, clever, c...
3                     [little, music, anything, speak]
4    [best, scene, movie, Gerardo, trying, find, so...
Name: comments, dtype: object

In [27]:
#use bag of words by applying the function and fit the data into it
bag_Of_Words_transformer = CountVectorizer(analyzer=removestopwords).fit(sentiments_data['comments'])


In [28]:
message_bagOfWords =bag_Of_Words_transformer.transform(sentiments_data['comments']) 


In [29]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagOfWords)

In [30]:
#print shape of the tfidf 
message_Tfid = tfidf_transformer.transform(message_bagOfWords)
print(message_Tfid.shape)

(748, 3259)


In [32]:
#choose naive Bayes model fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_model_detection = MultinomialNB().fit(message_Tfid,sentiments_data['labels'])

In [36]:
#check model for the predicted and expected value 
message = sentiments_data['comments'][0]

bag_of_word_for_message = bag_Of_Words_transformer.transform([message])
tfid = tfidf_transformer.transform(bag_of_word_for_message)
print('Predicted sentiments label :',spam_model_detection.predict(tfid)[0])
print('Expected sentiments label :',sentiments_data['labels'][0])

Predicted sentiments label : 0
Expected sentiments label : 0
